# GANN - CHECK SPELLING

## Setup

Load modules

In [1]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

# Clear any logs from previous runs
!rm -rf ./logs/ 

In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals

import time
import os
import datetime
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

Setup paths

In [3]:
data_path = '../data'

train_feature_path = data_path + '/tpm_combined.csv'
train_gene_name_path = data_path + '/tpm_combined_rows.csv'
train_cell_name_path = data_path + '/tpm_combined_cols.csv'

test_feature_path = data_path + '/tpm_combined_test.csv'
test_gene_name_path = data_path + '/tpm_combined_rows_test.csv'
test_cell_name_path = data_path + '/tpm_combined_cols_test.csv'

train_nonorm_path = data_path + '/tpm_combined_train_nonorm.csv'

## Load data

Load datasets into frames and check all the shapes match up

In [4]:
df_gene_names = pd.read_csv(train_gene_name_path, header=None)
df_cell_names = pd.read_csv(train_cell_name_path, header=None)
df_training_data = pd.read_csv(train_feature_path, header=None)

df_gene_names.columns = ['gene_name']

print(df_gene_names.shape)
print(df_cell_names.shape)
print(df_training_data.shape)

(6807, 1)
(1798, 1)
(6807, 1798)


In [5]:
df_training_data_nonorm = pd.read_csv(train_nonorm_path)
df_training_data_nonorm = df_training_data_nonorm.drop('gene_name', axis=1)

nonorm_max = df_training_data_nonorm.max().max()
nonorm_min = df_training_data_nonorm.min().min()
del df_training_data_nonorm

print(nonorm_max)
print(nonorm_min)

17.828987016884007
0.0


Load test data

In [6]:
df_gene_names_test = pd.read_csv(test_gene_name_path, header=None)
df_cell_names_test = pd.read_csv(test_cell_name_path, header=None)
df_test_data = pd.read_csv(test_feature_path, header=None)

print(df_gene_names_test.shape)
print(df_cell_names_test.shape)
print(df_test_data.shape)

(6807, 1)
(500, 1)
(6807, 500)


The number of genes in the input dataset determines the generator output as well as the dicriminator inputs

In [7]:
num_genes = df_gene_names.shape[0]
df_gene_names.shape

(6807, 1)

Take a look at the training data

In [8]:
df_training_data

,0,1,2,3,4,5,6,7,8,9,...,1788,1789,1790,1791,1792,1793,1794,1795,1796,1797
0,0.593519,0.724994,0.259323,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.397072,0.679657,0.541898,0.000000,0.000000,0.567361,0.442711,0.000000,0.000000
1,0.529515,0.730789,0.000000,0.291240,0.356175,0.000000,0.000000,0.252119,0.065884,0.734733,...,0.378425,0.347020,0.812647,0.487845,0.297422,0.516562,0.304545,0.488694,0.266055,0.000000
2,0.805992,0.015907,0.351573,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.551268,...,0.000000,0.661508,0.000000,0.605841,0.802402,0.681187,0.000000,0.000000,0.000000,0.000000
3,0.632745,0.442018,0.112515,0.506412,0.657094,0.628180,0.554082,0.738973,0.764555,0.726907,...,0.608894,0.586006,0.696437,0.427761,0.681078,0.672670,0.536335,0.451429,0.675192,0.527422
4,0.681206,0.236664,0.428852,0.420482,0.000000,0.000000,0.000000,0.000000,0.267296,0.625472,...,0.423746,0.111500,0.000000,0.622586,0.000000,0.611686,0.000000,0.502279,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6802,0.395655,0.749851,0.385655,0.000000,0.483893,0.697602,0.000000,0.000000,0.000000,0.747306,...,0.000000,0.455800,0.000000,0.420597,0.000000,0.407697,0.830038,0.741090,0.606057,0.000000
6803,0.000000,0.384173,0.265556,0.000000,0.000000,0.000000,0.000000,0.000000,0.444041,0.748770,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.557804,0.859781,0.868549,0.625063,0.571023
6804,0.430313,0.090124,0.605318,0.000000,0.000000,0.000000,0.000000,0.000000,0.520718,0.162545,...,0.000000,0.680744,0.128438,0.432505,0.656399,0.424025,0.275454,0.148875,0.000000,0.000000
6805,0.000000,0.000000,0.000000,0.621492,0.000000,0.649158,0.808326,0.000000,0.589434,0.000000,...,0.625170,0.000000,0.000000,0.000000,0.000000,0.491346,0.146496,0.000000,0.000000,0.000000


## Define model variables - COMMENT ON EACH ONE TO DESCRIBE

In [9]:
# Model params
LATENT_VARIABLE_SIZE = 100
GEN_L1_DENSE_SIZE = 600
GEN_L2_DENSE_SIZE = 600
GEN_L3_DENSE_SIZE = num_genes

DIS_INPUT_SIZE = num_genes
DIS_L1_DENSE_SIZE = 200
DIS_L2_DENSE_SIZE = 200

NOISE_STDEV = 0.1
POISSON_LAM = 1

# Training params
TRAIN_BATCH_SIZE = 10
TRAIN_BUFFER_SIZE = 10000
TEST_BATCH_SIZE = 500
TEST_BUFFER_SIZE = 500
GEN_BATCH_SIZE = 10
EPOCHS = 30

EX_GEN_BATCH_SIZE = 500

#LEARNING_RATE = 0.001
LEARNING_RATE = 1e-5

## Create training and test datasets

Create tensors from training data - Convert to Int32 for better work on GPU with batch and shuffle

In [10]:
train_dataset = tf.data.Dataset.from_tensor_slices(df_training_data.T.values.astype('float32')).shuffle(TRAIN_BUFFER_SIZE).batch(TRAIN_BATCH_SIZE)
print(train_dataset)

<BatchDataset shapes: (None, 6807), types: tf.float32>


In [11]:
test_dataset = tf.data.Dataset.from_tensor_slices(df_test_data.T.values.astype('float32')).shuffle(TEST_BUFFER_SIZE).batch(TEST_BATCH_SIZE)
print(test_dataset)

<BatchDataset shapes: (None, 6807), types: tf.float32>


## Define GANN model

Define function for contructing the generator

In [12]:
def create_generator():
    model = tf.keras.Sequential()
    
    #L1
    model.add(layers.Dense(GEN_L1_DENSE_SIZE, use_bias=False, input_shape=(LATENT_VARIABLE_SIZE,)))
    #model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    #assert model.output_shape == (None, GEN_L1_DENSE_SIZE, 1)  # Note: None is the batch size
    
    #L2
    model.add(layers.Dense(GEN_L2_DENSE_SIZE, use_bias=False))
    #model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    #assert model.output_shape == (None, GEN_L2_DENSE_SIZE, 1)
    
    #L3
    model.add(layers.Dense(GEN_L3_DENSE_SIZE, use_bias=False))
    #model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    #assert model.output_shape == (None, GEN_L3_DENSE_SIZE, 1)
    
    return model

Define function for constructing discriminator

In [13]:
def create_discriminator():
    model = tf.keras.Sequential()
    
    #L1
    model.add(layers.Dense(DIS_L1_DENSE_SIZE, use_bias=False, input_shape=(DIS_INPUT_SIZE,)))
    model.add(layers.LeakyReLU())
    #model.add(layers.Dropout(0.3))
    
    #L2
    model.add(layers.Dense(DIS_L2_DENSE_SIZE, use_bias=False))
    model.add(layers.LeakyReLU())
    #model.add(layers.Dropout(0.3))
    
    #L3
    model.add(layers.Flatten())
    model.add(layers.Dense(1))
    
    return model

Define the noise generation function

In [14]:
def gen_noise(batch_size):
    # Create some random noise for the generator
    n_noise = tf.random.normal([batch_size, LATENT_VARIABLE_SIZE], mean=0.0, stddev=NOISE_STDEV)
    p_noise = tf.random.poisson([batch_size, LATENT_VARIABLE_SIZE], lam=POISSON_LAM)
    noise = tf.abs(n_noise + p_noise)
    return noise

Define the loss functions

In [15]:
# This method returns a helper function to compute cross entropy loss
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

In [16]:
def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss
    
    #total_loss = tf.reduce_mean(real_output) - tf.reduce_mean(fake_output)
    #return total_loss

In [17]:
def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)
    #total_loss = -tf.reduce_mean(fake_output)
    #return total_loss

Util functions

In [18]:
def data_frame_from_gen(profile, label):
    # Create formatted dataframe from generator result
    df_gen_prof = pd.DataFrame(generated_profile.numpy()).T
    df_gen_prof = df_gene_names.join(df_gen_prof, lsuffix='', rsuffix='', how='inner')
    df_gen_prof.index = df_gen_prof.gene_name
    df_gen_prof = df_gen_prof.drop('gene_name', axis=1)
    df_gen_prof = df_gen_prof.add_prefix(label)

    # Get limits
    gen_min = df_gen_prof.min().min()
    gen_max = df_gen_prof.max().max()

    # Scale everything up to 0
    df_gen_prof = df_gen_prof + (gen_min*-1)
    gen_max = df_gen_prof.max().max()
    gen_min = df_gen_prof.min().min()

    # Rescale to between real world min maxes
    df_gen_prof = df_gen_prof / gen_max
    df_gen_prof = df_gen_prof * nonorm_max
    
    return df_gen_prof

## Define the training loops

In [19]:
# Input is a batch of real cell profiles from the training set
# @tf.function
def train_step(cell_profiles):
    noise = gen_noise(GEN_BATCH_SIZE)
    
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_profiles = generator(noise, training=True)
        
        real_output = discriminator(cell_profiles, training=True)
        fake_output = discriminator(generated_profiles, training=True)

        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)
        
        met_gen_loss(gen_loss)
        met_disc_loss(disc_loss)
        
    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))
    
    return

## Create GANN model

Create generator and discriminator

In [20]:
generator = create_generator()
discriminator = create_discriminator()

Define optimizer

In [21]:
generator_optimizer = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE, beta_1=0.9, beta_2=0.999, epsilon=1e-07)
discriminator_optimizer = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE, beta_1=0.9, beta_2=0.999, epsilon=1e-07)

## Create checkpoints

In [22]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generator,
                                 discriminator=discriminator)

## Generate from test data to check network

In [23]:
# Generate a single test set
noise = gen_noise(EX_GEN_BATCH_SIZE)
generated_profile = generator(noise, training=False)
df_gen_prof_1 = data_frame_from_gen(generated_profile, 'gencell_ep0_')

# Vis
df_gen_prof_1

# Save to file
#df_gen_prof_1.to_csv(data_path + '/gen_prof_pre.csv')

,gencell_ep0_0,gencell_ep0_1,gencell_ep0_2,gencell_ep0_3,gencell_ep0_4,gencell_ep0_5,gencell_ep0_6,gencell_ep0_7,gencell_ep0_8,gencell_ep0_9,...,gencell_ep0_490,gencell_ep0_491,gencell_ep0_492,gencell_ep0_493,gencell_ep0_494,gencell_ep0_495,gencell_ep0_496,gencell_ep0_497,gencell_ep0_498,gencell_ep0_499
gene_name,,,,,,,,,,,,,,,,,,,,,
Rfc3,8.945637,5.733241,3.860522,5.619954,5.586335,6.970235,4.338315,4.549016,5.009731,4.163322,...,5.424246,4.402611,4.928729,4.162288,3.108255,3.963334,4.221949,5.123520,5.077371,4.183359
Cd47,3.738709,6.256884,4.448080,4.054791,3.151270,5.373478,4.040785,3.867641,3.853901,3.849575,...,4.048962,3.853764,3.765724,4.609506,3.302379,3.876153,4.352907,4.469342,3.901982,4.072795
Elmo2,5.200176,5.034015,4.367820,4.389033,3.275808,4.553952,4.151926,4.272325,5.511501,5.136298,...,6.041917,7.925032,4.330449,5.032185,6.416893,3.790214,4.398872,4.268808,6.591664,6.079050
Crip2,6.406047,6.777477,4.845940,5.980703,6.169468,3.798367,5.310357,5.590196,5.810795,4.467186,...,6.990990,5.949615,4.702023,6.335579,6.546120,4.798983,6.166876,5.177269,5.966042,4.368247
Pprc1,3.406488,4.168306,3.645792,4.300521,4.956060,4.561098,4.906742,3.943385,5.413904,3.865657,...,4.532197,5.760712,3.690476,4.809052,5.970742,6.596510,4.736276,4.186600,4.369345,5.713038
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Hprt,3.915553,3.186345,3.832254,2.772286,2.920236,3.778947,3.616331,3.301115,3.704352,4.052122,...,4.769273,4.114681,3.481516,4.099298,4.000013,3.558737,3.259285,3.562539,3.231498,2.215208
Atraid,5.856579,8.187841,4.302191,3.987302,5.923300,8.318023,5.480140,6.849266,3.915834,4.580674,...,9.347400,4.448579,5.678010,4.647724,4.519629,5.084782,7.586717,4.147527,9.219702,4.389924
Chek2,5.438097,8.147394,3.737049,3.643571,4.227227,6.925369,6.109890,4.225889,4.986666,4.894701,...,4.560132,4.798263,4.286907,5.410535,5.647682,6.589124,6.027516,9.764347,4.140214,4.399301


## Train the GANN

Define tensorboard metrics

In [24]:
met_gen_loss = tf.keras.metrics.Mean('gen_loss', dtype=tf.float32)
met_disc_loss = tf.keras.metrics.Mean('disc_loss', dtype=tf.float32)
met_test_loss = tf.keras.metrics.Mean('test_loss', dtype=tf.float32)

Create log directories

In [25]:
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

gen_log_dir = 'logs/gradient_tape/' + current_time + '/gen_train'
disc_log_dir = 'logs/gradient_tape/' + current_time + '/disc_train'
test_log_dir = 'logs/gradient_tape/' + current_time + '/disc_test'
all_log_dir = 'logs/gradient_tape/' + current_time + '/all'

all_summary_writer = tf.summary.create_file_writer(all_log_dir)
gen_summary_writer = tf.summary.create_file_writer(gen_log_dir)
disc_summary_writer = tf.summary.create_file_writer(disc_log_dir)
test_summary_writer = tf.summary.create_file_writer(test_log_dir)

Run the training model

In [26]:
print('Running...')

for epoch in range(EPOCHS):
    
    # Save checkpoints and gen example data
    if epoch % 10 == 0:   
        checkpoint.save(file_prefix = checkpoint_prefix)
        
        # Generate a profile set
        noise = gen_noise(EX_GEN_BATCH_SIZE)
        generated_profile = generator(noise, training=False)
        df_gen_prof = data_frame_from_gen(generated_profile, 'gencell_ep' + str(epoch) + '_')
        df_gen_prof.to_csv(data_path + '/gen_prof_' + str(epoch) + '.csv')
    
    # Logging
    start = time.time()
    
    #Train the epoch
    for data_batch in train_dataset:
        train_step(data_batch)
        
    #Run test data through discriminator
    for data_batch in test_dataset:
        test_decision = discriminator(data_batch, training=False)

    test_loss = cross_entropy(tf.ones_like(test_decision), test_decision)
    met_test_loss(test_loss)
    
    #Log metrics
    with all_summary_writer.as_default():
        tf.summary.scalar('2_gen_loss', met_gen_loss.result(), step=epoch)
        tf.summary.scalar('3_disc_loss', met_disc_loss.result(), step=epoch)
        tf.summary.scalar('3_test_loss', met_test_loss.result(), step=epoch)
    
    with gen_summary_writer.as_default():
        tf.summary.scalar('1_loss', met_gen_loss.result(), step=epoch)
           
    with disc_summary_writer.as_default():
        tf.summary.scalar('1_loss', met_disc_loss.result(), step=epoch)
    
    with test_summary_writer.as_default():
        tf.summary.scalar('1_loss', met_test_loss.result(), step=epoch)

    # Logging
    #print ('Time for epoch {} is {} sec.'.format(epoch + 1, time.time()-start))
    time.time()
      
    #Log stats
    template = 'Epoch {}, Gen_loss: {}, Disc_loss: {}, Test_loss: {}'
    print (template.format(epoch+1,
                           met_gen_loss.result(), 
                           met_disc_loss.result(),
                           met_test_loss.result()))
    
    # Reset metrics every epoch
    met_gen_loss.reset_states()
    met_disc_loss.reset_states()
    met_test_loss.reset_states()
    
# Generate a profile set
noise = gen_noise(EX_GEN_BATCH_SIZE)
generated_profile = generator(noise, training=False)
df_gen_prof = data_frame_from_gen(generated_profile, 'gencell_ep' + str(EPOCHS) + '_')
df_gen_prof.to_csv(data_path + '/gen_prof_' + str(EPOCHS) + '.csv')

Running...
Epoch 1, Gen_loss: 0.2454620599746704, Disc_loss: 1.8153274059295654, Test_loss: 0.17771019041538239
Epoch 2, Gen_loss: 0.2363108992576599, Disc_loss: 1.8622262477874756, Test_loss: 0.4955480098724365
Epoch 3, Gen_loss: 0.34907957911491394, Disc_loss: 1.979713797569275, Test_loss: 0.9451809525489807
Epoch 4, Gen_loss: 0.5352790355682373, Disc_loss: 1.7421742677688599, Test_loss: 0.7766973972320557
Epoch 5, Gen_loss: 0.5957620143890381, Disc_loss: 1.5735509395599365, Test_loss: 0.6916200518608093
Epoch 6, Gen_loss: 0.6484591364860535, Disc_loss: 1.3220919370651245, Test_loss: 0.4871649742126465
Epoch 7, Gen_loss: 0.7735310792922974, Disc_loss: 1.0548357963562012, Test_loss: 0.3853340744972229
Epoch 8, Gen_loss: 0.8803747296333313, Disc_loss: 0.9373819828033447, Test_loss: 0.41591861844062805
Epoch 9, Gen_loss: 0.8895942568778992, Disc_loss: 1.067600131034851, Test_loss: 0.7159780859947205
Epoch 10, Gen_loss: 0.9113759398460388, Disc_loss: 1.344366431236267, Test_loss: 0.89159

Start tensorboard

In [27]:
#%tensorboard --logdir {train_log_dir} --host localhost --port 6006